In [321]:
# Data handling libraries
import pandas as pd
pd.options.display.max_rows = 10000
import numpy as np
import boto3
import io
import requests as req

# File manipulation libraries
import os
import pickle

# Data visualization libraries
import matplotlib.pyplot as plt
%matplotlib inline

# Initialize S3 client, location of files for this project
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/blog_data/GHG-GDP_Divergence_D3/"

RAW_DATA = s3_folder + "Raw_Data/"
PROCESSED_DATA = s3_folder + "Processed_Data/"
FINAL_DATA = s3_folder + "Final_Data/"
CONVERSIONS = s3_folder + "Conversions/"

# Functions for reading and uploading data to/from S3
def read_from_S3(bucket, key, index_col=0):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=[index_col], encoding="utf8")
    return(df)

def write_to_S3(df, bucket, key):
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer)
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())

Load Raw Data from S3

In [158]:
# These four files are derived from the original CDIAC data sheet
# They were initially cleaned (using code outlined at the bottom of this notebook)
# And then uploaded to Amazon S3
file_names = ['Territorial Emissions GCB', 
              'Consumption Emissions GCB', 
              'Emissions Transfers GCB', 
              'Territorial Emissions CDIAC']

# Initialize a dictionary to store the raw data
cdiac_raw_data = {}

# Load each of the raw datasets from S3
# Reference: https://stackoverflow.com/questions/37703634/how-to-import-a-text-file-on-aws-s3-into-pandas-without-writing-to-disk
for file in file_names:
    cdiac_raw_data[file] = read_from_S3(s3_bucket, RAW_DATA+file+".csv")

In [144]:
#cdiac_raw_data["Territorial Emissions GCB"].head()
#cdiac_raw_data["Consumption Emissions GCB"].head()
#cdiac_raw_data["Emissions Transfers GCB"].head()
#cdiac_raw_data["Territorial Emissions CDIAC"].head()

Convert raw data to pct_change data for territorial and consumption emissions, load to S3

In [358]:
# Convert raw data to percent change by year from 2000 forward
territorial_emissions_abs_raw = cdiac_raw_data["Territorial Emissions GCB"]
territory_gcb_pct_change = territorial_emissions_abs_raw.loc[1999:2015].transpose().pct_change(axis=1).loc[:,2000:]
territory_gcb_abs_val = territorial_emissions_abs_raw.loc[2000:2015].transpose()

consumption_emissions_abs_raw = cdiac_raw_data["Consumption Emissions GCB"]
consumption_gcb_pct_change = consumption_emissions_abs_raw.loc[1999:2015].transpose().pct_change(axis=1).loc[:,2000:]
consumption_gcb_abs_val = consumption_emissions_abs_raw.loc[2000:2015].transpose()

# Upload these percent change figures to S3

write_to_S3(territory_gcb_pct_change, s3_bucket, PROCESSED_DATA + \
                   "Territorial Emissions GCB percent changes 2000-2015.csv")
write_to_S3(territory_gcb_abs_val, s3_bucket, PROCESSED_DATA + \
                   "Territorial Emissions GCB absolute values 2000-2015.csv")

write_to_S3(consumption_gcb_pct_change, s3_bucket, PROCESSED_DATA + \
                   "Consumption Emissions GCB percent changes 2000-2015.csv")
write_to_S3(consumption_gcb_abs_val, s3_bucket, PROCESSED_DATA + \
                   "Consumption Emissions GCB absolute values 2000-2015.csv")

Download Conversions used to align CDIAC, World Bank, and ISO3 country designations

In [161]:
# CDIAC names to World Bank names
cdiac_to_wb_name_conversion = read_from_S3(s3_bucket, CONVERSIONS+"CDIAC to World Bank name conversion.csv")
# World Bank names to ISO3 codes
wb_name_to_iso3_conversion = read_from_S3(s3_bucket, CONVERSIONS+"World Bank to ISO3 name conversion.csv")

Create final data for the D3 application by adding ISO3 codes to the CDIAC pct change data

In [368]:
# Download pct_change data from S3
territory_gcb_pct_change = read_from_S3(s3_bucket, PROCESSED_DATA+"Territorial Emissions GCB percent changes 2000-2015.csv")
consumption_gcb_pct_change = read_from_S3(s3_bucket, PROCESSED_DATA+"Consumption Emissions GCB percent changes 2000-2015.csv")

territory_gcb_abs_val = read_from_S3(s3_bucket, PROCESSED_DATA+"Territorial Emissions GCB absolute values 2000-2015.csv")
consumption_gcb_abs_val = read_from_S3(s3_bucket, PROCESSED_DATA+"Consumption Emissions GCB absolute values 2000-2015.csv")

dfs = {"territory_pct":territory_gcb_pct_change,
        "consumption_pct":consumption_gcb_pct_change,
         "territory_abs":territory_gcb_abs_val,
         "consumption_abs":consumption_gcb_abs_val}


# Name for Congo didn't match in the CDIAC data and crosswalk file
def replace_congo(name):
    if name == "Congo":
        return("Congo (Rep)")
    else:
        return(name)
    
# Add the wb_name to each dataframe
def fetch_name(name):
    try:
        return(cdiac_to_wb_name_conversion.loc[name][0])
    except:
        return(np.nan)
    
def add_iso(name):
    try:
        return(wb_name_to_iso3_conversion.loc[name,"ISO"])
    except:
        return(np.nan)

def create_summary_values_CDIAC(row, is_consumption_data=True):
    if is_consumption_data:
        val = row["2014"] - row["2000"]
        return(val)
    else:
        val = row["2015"] - row["2000"]
        return(val)

for df_name, df in dfs.items():
    print(df_name)
    df.index = list(map(replace_congo, df.index))
    df["Country Name"] = list(map(fetch_name, df.index)) 
    df = df.loc[pd.notnull(df["Country Name"])]
    df = df.set_index("Country Name")
    df["ISO"] = list(map(add_iso, df.index))
    if("consumption" in df_name):
        df["2000-2014"] = df.apply(lambda row: create_summary_values_CDIAC(row, True), axis=1)
    else:
        df["2000-2015"] = df.apply(lambda row: create_summary_values_CDIAC(row, False), axis=1)
    
    dfs[df_name] = df
# territory_gcb_pct_change.index = list(map(replace_congo, territory_gcb_pct_change.index))
# consumption_gcb_pct_change.index = list(map(replace_congo, consumption_gcb_pct_change.index))
    
# territory_gcb_pct_change["Country Name"] = list(map(fetch_name, territory_gcb_pct_change.index)) #apply(lambda row: fetch_name(row.name), axis=1)
# consumption_gcb_pct_change["Country Name"] = list(map(fetch_name, consumption_gcb_pct_change.index)) # consumption_gcb_pct_change.apply(lambda row: fetch_name(row.name), axis=1)

# # Only keep the CDIAC data where there is a matching world bank country
# territory_gcb_pct_change = territory_gcb_pct_change.loc[pd.notnull(territory_gcb_pct_change["Country Name"])]
# consumption_gcb_pct_change = consumption_gcb_pct_change.loc[pd.notnull(consumption_gcb_pct_change["Country Name"])]

# # Set index to be World Bank name
# territory_gcb_pct_change = territory_gcb_pct_change.set_index("Country Name")
# consumption_gcb_pct_change = consumption_gcb_pct_change.set_index("Country Name")

# # Use wb_name to 

# territory_gcb_pct_change["ISO"] = list(map(add_iso, territory_gcb_pct_change.index))
# consumption_gcb_pct_change["ISO"] = list(map(add_iso, consumption_gcb_pct_change.index))



territory_pct
consumption_pct
territory_abs
consumption_abs


In [373]:
# All countries have an assigned iso if these comes up empty dataframes
print(dfs["territory_pct"].loc[pd.isnull(dfs["territory_pct"]["ISO"])])
print(dfs["consumption_pct"].loc[pd.isnull(dfs["consumption_pct"]["ISO"])])
print(dfs["territory_abs"].loc[pd.isnull(dfs["territory_abs"]["ISO"])])
print(dfs["consumption_abs"].loc[pd.isnull(dfs["consumption_abs"]["ISO"])])

Empty DataFrame
Columns: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, ISO, 2000-2015]
Index: []
Empty DataFrame
Columns: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, ISO, 2000-2014]
Index: []
Empty DataFrame
Columns: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, ISO, 2000-2015]
Index: []
Empty DataFrame
Columns: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, ISO, 2000-2014]
Index: []


In [379]:
# Export final files
write_to_S3(dfs["territory_pct"], s3_bucket, FINAL_DATA + "Territory Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv")
write_to_S3(dfs["consumption_pct"], s3_bucket, FINAL_DATA + "Consumption Emissions GCB percent changes with ISO3 2000-2015 plus summary data.csv")
write_to_S3(dfs["territory_abs"], s3_bucket, FINAL_DATA + "Territory Emissions GCB absolute values with ISO3 2000-2015 plus summary data.csv")
write_to_S3(dfs["consumption_abs"], s3_bucket, FINAL_DATA + "Consumption Emissions GCB absolute values with ISO3 2000-2015 plus summary data.csv")

In [381]:
# Territory or Consumption?
emissions_type = "Territory"
# absolute values or percent changes?
metric = "absolute values"

df = read_from_S3(s3_bucket, FINAL_DATA + \
                  "{} Emissions GCB {} with ISO3 2000-2015 plus summary data.csv".format(emissions_type,metric) \
                  , index_col="ISO")
df

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2000-2015
ISO,,,,,,,,,,,,,,,,,,
ALB,Albania,0.824000,0.879000,1.023000,1.171000,1.136000,1.160000,1.063000,1.071000,1.193000,1.194000,1.254000,1.429000,1.285000,1.313000,1.308100,1.323801,0.499801
DZA,Algeria,23.979000,22.987000,24.776000,25.234000,24.405000,29.214000,27.588000,29.860000,30.079000,33.115000,32.500000,33.048000,35.448000,36.601000,39.257647,40.921606,16.942606
AND,Andorra,0.143000,0.143000,0.145000,0.146000,0.154000,0.157000,0.149000,0.147000,0.147000,0.141000,0.141000,0.134000,0.134000,0.134000,0.134952,0.135947,-0.007053
AGO,Angola,2.602000,2.654000,3.454000,2.472000,5.125000,5.224000,6.072000,6.859000,7.011000,7.579000,7.924000,8.274000,9.108000,8.853000,8.848174,8.990173,6.388173
ATG,Antigua and Barbuda,0.094000,0.094000,0.099000,0.106000,0.111000,0.112000,0.116000,0.128000,0.131000,0.139000,0.143000,0.140000,0.143000,0.143000,0.146480,0.149771,0.055771
ARG,Argentina,38.761000,36.466000,34.010000,36.832000,42.975000,44.208000,47.842000,47.771000,51.570000,49.076000,51.246000,52.259000,52.457000,51.764000,52.080937,52.799821,14.038821
ARM,Armenia,0.945000,0.966000,0.830000,0.935000,0.994000,1.187000,1.195000,1.381000,1.516000,1.189000,1.150000,1.341000,1.553000,1.499000,1.490023,1.525120,0.580120
AUS,Australia,95.492749,97.612808,98.761296,100.531107,104.126436,105.235025,106.750573,108.847285,110.326918,111.476113,110.862716,110.181640,110.934183,108.328039,107.294472,109.099849,13.607100
AUT,Austria,18.088085,19.186537,19.685205,21.294747,21.394377,21.721753,20.997530,20.269681,20.214589,18.472359,19.795804,19.194096,18.476708,18.547249,17.538909,18.161297,0.073212


Process GDP and other WB indicator Data

In [257]:
data_names_and_codes = {'EG.ELC.ACCS.ZS': 'Access to electricity (% of population)',
 'EG.FEC.RNEW.ZS': 'Renewable energy consumption (% of total final energy consumption)',
 'IT.NET.USER.ZS': 'Individuals using the Internet (% of population)',
 'NE.CON.PRVT.PC.KD': 'Household final consumption expenditure per capita (constant 2010 US$)',
 'NV.IND.TOTL.KD': 'Industry, value added (constant 2010 US$)',
 'NY.GDP.TOTL.RT.ZS': 'Total natural resources rents (% of GDP)',
 'SG.GEN.PARL.ZS': 'Proportion of seats held by women in national parliaments (%)',
 'SL.EMP.TOTL.SP.ZS': 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)',
 'SM.POP.NETM': 'Net migration',
 'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)',
 'SP.URB.TOTL.IN.ZS': 'Urban population (% of total)',
 'TM.VAL.MRCH.CD.WT': 'Merchandise imports (current US$)',
 'NY.GDP.MKTP.CD': 'GDP (current US$)'}

column_long_name_to_short_name = {
    'Renewable energy consumption (% of total final energy consumption)': 'renewable_energy_consumption_of_total_final_energy_consumpti',
    'Household final consumption expenditure per capita (constant 2010 US$)': 'household_final_consumption_expenditure_per_capita_constant_20',
    'Merchandise imports (current US$)': 'merchandise_imports_current_us_tm_val_mrch_cd_wt',
    'Industry, value added (constant 2010 US$)': 'industry_value_added_constant_2010_us_nv_ind_totl_kd',
    'Access to electricity (% of population)': 'access_to_electricity_of_population_eg_elc_accs_zs',
    'Urban population (% of total)': 'urban_population_of_total_sp_urb_totl_in_zs',
    'Employment to population ratio, 15+, total (%) (modeled ILO estimate)': 'employment_to_population_ratio_15_total_modeled_ilo_est',
    'Total natural resources rents (% of GDP)': 'total_natural_resources_rents_of_gdp_ny_gdp_totl_rt_zs',
    'Life expectancy at birth, total (years)': 'life_expectancy_at_birth_total_years_sp_dyn_le00_in',
    'Net migration': 'net_migration_sm_pop_netm',
    'Proportion of seats held by women in national parliaments (%)': 'proportion_of_seats_held_by_women_in_national_parliaments',
    'Individuals using the Internet (% of population)': 'individuals_using_the_internet_of_population_it_net_user_z',
    'GDP (current US$)': 'GDP'
}

series_code_to_data_viz_name = {}
for key, value in data_names_and_codes.items():
    series_code_to_data_viz_name[key] = column_long_name_to_short_name[value]
    
series_code_to_data_viz_name 

{'EG.ELC.ACCS.ZS': 'access_to_electricity_of_population_eg_elc_accs_zs',
 'EG.FEC.RNEW.ZS': 'renewable_energy_consumption_of_total_final_energy_consumpti',
 'IT.NET.USER.ZS': 'individuals_using_the_internet_of_population_it_net_user_z',
 'NE.CON.PRVT.PC.KD': 'household_final_consumption_expenditure_per_capita_constant_20',
 'NV.IND.TOTL.KD': 'industry_value_added_constant_2010_us_nv_ind_totl_kd',
 'NY.GDP.MKTP.CD': 'GDP',
 'NY.GDP.TOTL.RT.ZS': 'total_natural_resources_rents_of_gdp_ny_gdp_totl_rt_zs',
 'SG.GEN.PARL.ZS': 'proportion_of_seats_held_by_women_in_national_parliaments',
 'SL.EMP.TOTL.SP.ZS': 'employment_to_population_ratio_15_total_modeled_ilo_est',
 'SM.POP.NETM': 'net_migration_sm_pop_netm',
 'SP.DYN.LE00.IN': 'life_expectancy_at_birth_total_years_sp_dyn_le00_in',
 'SP.URB.TOTL.IN.ZS': 'urban_population_of_total_sp_urb_totl_in_zs',
 'TM.VAL.MRCH.CD.WT': 'merchandise_imports_current_us_tm_val_mrch_cd_wt'}

In [337]:
indicators = series_code_to_data_viz_name

# ['EG.FEC.RNEW.ZS', 'NE.CON.PRVT.PC.KD', 'TM.VAL.MRCH.CD.WT',
#        'NV.IND.TOTL.KD', 'EG.ELC.ACCS.ZS', 'SP.URB.TOTL.IN.ZS',
#        'SL.EMP.TOTL.SP.ZS', 'NY.GDP.TOTL.RT.ZS', 'SP.DYN.LE00.IN',
#        'SM.POP.NETM', 'SG.GEN.PARL.ZS', 'IT.NET.USER.ZS']

all_world_bank_data = pd.DataFrame(columns=['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       'ISO', 'Indicator'])
for indicator in indicators:
    # Results are paginated
    res = req.get("http://api.worldbank.org/countries/all/indicators/{}?date=1999:2016&format=json&per_page=10000".format(indicator))
    data = pd.io.json.json_normalize(res.json()[1])
    data = data[["country.value", "date", "value"]]
    value_name = series_code_to_data_viz_name[indicator]
    data.columns = ["Country Name", "Year", value_name]
    data = data.pivot(index="Country Name", columns="Year", values=value_name).astype(float)
    data["ISO"] = list(map(add_iso, data.index))
    data = data.loc[pd.notnull(data["ISO"])]
    data["Indicator"] = value_name 
    all_world_bank_data = all_world_bank_data.append(data)
    
all_world_bank_data.index.name = "Country Name"
all_world_bank_data.reset_index(inplace=True)
all_world_bank_data.columns.name = ""

In [335]:
reverse_map = {v: k for k, v in column_long_name_to_short_name.items()}
def create_summary_values_World_Bank(row):
    #print(row)
    indicator = reverse_map[row["Indicator"]]
    
    if indicator == 'Renewable energy consumption (% of total final energy consumption)':
        val = row["2014"] - row["2000"]
        return(val)

    elif indicator == 'GDP':
        val = row["2015"] - row["2000"]
        return(val)

    elif indicator == 'Household final consumption expenditure per capita (constant 2010 US$)':
        val = row["2015"]
        return(val)

    elif indicator == 'Merchandise imports (current US$)':
        val = row["2015"] - row["2000"]
        return(val)

    elif indicator == 'Industry, value added (constant 2010 US$)':
        val = row["2015"] - row["2000"]
        return(val)

    elif indicator == 'Access to electricity (% of population)':
        val = row["2014"]
        return(val)

    elif indicator == 'Urban population (% of total)':
        val = row["2015"]
        return(val)

    elif indicator == 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)':
        val = row["2015"] - row["2000"]
        return(val)

    elif indicator == 'Total natural resources rents (% of GDP)':
        val = row["2015"]
        return(val)

    elif indicator == 'Life expectancy at birth, total (years)':
        val = row["2015"]
        return(val)

    elif indicator == 'Net migration':
        val = row["2012"]
        return(val)

    elif indicator == 'Proportion of seats held by women in national parliaments (%)':
        val = row["2015"]
        return(val)

    elif indicator == 'Individuals using the Internet (% of population)':
        val = row["2015"]
        return(val)
     
all_world_bank_data["2000-2015"] = all_world_bank_data.apply(create_summary_values_World_Bank, axis=1)

In [338]:
all_world_bank_data.head()

,Country Name,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2009,2010,2011,2012,2013,2014,2015,2016,ISO,Indicator
0,Afghanistan,0.016871,0.163713,1.006099,3.467205,7.209179,13.969172,23.000000,27.506411,34.290512,...,47.888466,42.700000,61.514420,69.100000,75.154373,89.5,NaN,NaN,AFG,access_to_electricity_of_population_eg_elc_acc...
1,Albania,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,NaN,ALB,access_to_electricity_of_population_eg_elc_acc...
2,Algeria,96.389084,96.702133,97.004044,97.298698,97.590019,97.881889,98.184265,98.490738,98.806519,...,99.443893,99.711174,99.889542,99.973083,99.996918,100.0,NaN,NaN,DZA,access_to_electricity_of_population_eg_elc_acc...
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASM,access_to_electricity_of_population_eg_elc_acc...
4,Andorra,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,NaN,NaN,AND,access_to_electricity_of_population_eg_elc_acc...


In [341]:
write_to_S3(all_world_bank_data, s3_bucket, PROCESSED_DATA + "World Bank Data with ISO3, 1999-2016 with 2000-2015 Summary Values.csv")

Calculate Index Values

In [354]:
# Calculating index values
# formula = (1 – ΔCO2)*(1 + ΔGDP) - ΔCO2 + ΔGDP

world_bank_data = read_from_S3(s3_bucket, PROCESSED_DATA + "World Bank Data with ISO3, 1999-2016 with 2000-2015 Summary Values.csv")
tuples = list(zip(*[world_bank_data["Indicator"],world_bank_data["Country Name"]]))
multi_index = pd.MultiIndex.from_tuples(tuples, names=["Indicator", "Country Name"])
world_bank_data.index = multi_index

gdp_data = world_bank_data.loc["GDP"]
gdp_data.set_index(["ISO"], inplace=True)

year_columns = [str(yr) for yr in range(1999,2016)]
gdp_data_just_years = gdp_data[year_columns]
gdp_per_change_data = gdp_data_just_years.pct_change(axis=1)

#print(gdp_per_change_data.head())

def calc_index(co2, gdp):
    return((1-co2)*(1+gdp) - co2 + gdp)
    
# import CO2 change
## Territorial
territory_emissions = read_from_S3(s3_bucket, FINAL_DATA + \
                   "Territory Emissions GCB percent changes with ISO3 2000-2015.csv", index_col = "ISO")

## Consumption
consumption_emissions = read_from_S3(s3_bucket, FINAL_DATA + \
                   "Consumption Emissions GCB percent changes with ISO3 2000-2015.csv", index_col = "ISO")

# https://stackoverflow.com/questions/22149584/what-does-axis-in-pandas-mean
territory_emissions_gdp_index = calc_index(territory_emissions.drop("Country Name", axis=1), gdp_per_change_data)
consumption_emissions_gdp_index = calc_index(consumption_emissions.drop("Country Name", axis=1), gdp_per_change_data)

In [357]:
write_to_S3(territory_emissions_gdp_index, s3_bucket, FINAL_DATA + \
                   "ICGGD calculated with Territory Emissions.csv")

write_to_S3(consumption_emissions_gdp_index, s3_bucket, FINAL_DATA + \
                   "ICGGD calculated with Consumption Emissions.csv")

In [ ]:
### FOR HISTORICAL PURPOSES - I USED THE BELOW TO FORMAT THE DATA ###

In [3]:
os.chdir("/Users/nathansuberi/Desktop/WRI_Programming/Py_Scripts/Data Packs/Materials for Nate")

# Reference from above:
!mkdir temp

os.chdir("temp")

# In order for this to be work, sheet needs to be public
# Or "On, anyone with link can access"
# CDIAC Data
!curl "https://docs.google.com/spreadsheets/d/1vd7NWFmpXJsHNNERowemLEvSoQkpKYFF-AxshdQ3GIE/export#gid=318056468?format=xls" > cdiac_data.xls

dest = os.getcwd()
cdiac_data = pd.ExcelFile(dest + "/cdiac_data.xls")

os.chdir("..")
!rm -r temp

# cdiac_data

sheet_names = cdiac_data.sheet_names
cdiac_dataframes = {}
for name in sheet_names:
    cdiac_dataframes[name] = cdiac_data.parse(name)   
    
##### When you have the excel spreadsheet on your local system,
# access a specific sheet like so:
#list_names = cdiac_data.sheet_names
#zipped = zip(list_names, [0]*len(list_names))

#sheet_names = dict(zipped)

#print(sheet_names)

# For some strange reason, these only work 
# if I don't make the dict above
#a, b = zip(*zipped)
#print(a)
#print(b)

cdiac_raw_data = {}

curr = "Territorial Emissions GCB"
tmp = cdiac_dataframes[curr]

# Remove first informational rows
tmp = tmp.iloc[14:tmp.shape[0],:]

# Replace nan in first row with "Year", set new index
new_index = ["Year"] + list(tmp.index)[1:]
tmp.index = new_index

# Make columns equal to country names
tmp.columns = tmp.iloc[0]
# Drop the row that had the country names
tmp = tmp.drop(["Year"])

# Drops all rows and columns that are completely null
tmp = tmp.dropna(axis=0, how="all")
tmp = tmp.dropna(axis=1, how="all")

# Remove three nan columns at end
# Remove summary rows at bottom
tmp = tmp.iloc[:-2,:-3]

tmp

cdiac_raw_data[curr] = tmp




curr = "Consumption Emissions GCB"
tmp = cdiac_dataframes[curr]

# Remove first informational rows
tmp = tmp.iloc[7:tmp.shape[0],:]

# Replace nan in first row with "Year", set new index
new_index = ["Year"] + list(tmp.index)[1:]
tmp.index = new_index

# Make columns equal to country names
tmp.columns = tmp.iloc[0]
# Drop the row that had the country names
tmp = tmp.drop(["Year"])

# Drops all rows and columns that are completely null
tmp = tmp.dropna(axis=0, how="all")
tmp = tmp.dropna(axis=1, how="all")

# Remove three nan columns at end
# Remove summary rows at bottom
tmp = tmp.iloc[:-2,:-3]

tmp

cdiac_raw_data[curr] = tmp




curr = "Emissions Transfers GCB"
tmp = cdiac_dataframes[curr]

# Remove first informational rows
tmp = tmp.iloc[7:tmp.shape[0],:]

# Replace nan in first row with "Year", set new index
new_index = ["Year"] + list(tmp.index)[1:]
tmp.index = new_index

# Make columns equal to country names
tmp.columns = tmp.iloc[0]
# Drop the row that had the country names
tmp = tmp.drop(["Year"])

# Drops all rows and columns that are completely null
tmp = tmp.dropna(axis=0, how="all")
tmp = tmp.dropna(axis=1, how="all")

# No need to remove anything
# tmp = tmp.iloc[:,:]

tmp

cdiac_raw_data[curr] = tmp





curr = "Territorial Emissions CDIAC"
tmp = cdiac_dataframes[curr]

# Remove first informational rows
tmp = tmp.iloc[13:tmp.shape[0],:]

# Replace nan in first row with "Year", set new index
new_index = ["Year"] + list(tmp.index)[1:]
tmp.index = new_index

# Make columns equal to country names
tmp.columns = tmp.iloc[0]
# Drop the row that had the country names
tmp = tmp.drop(["Year"])

# Drops all rows and columns that are completely null
tmp = tmp.dropna(axis=0, how="all")
tmp = tmp.dropna(axis=1, how="all")

# Remove one nan columns at end
# Remove 8 summary rows from end
tmp = tmp.iloc[:-8,:-1]

tmp

cdiac_raw_data[curr] = tmp



file_dest = os.getcwd()
#pickle.dump(cdiac_dataframes_clean, open(file_dest + "/clean_cdiac_dataframes.pkl", 'wb'))

for file in cdiac_dataframes_clean:
    cdiac_raw_data[file].to_csv(file_dest+"/"+file+".csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1428k    0 1428k    0     0   624k      0 --:--:--  0:00:02 --:--:--  624k


In [ ]:
# Original processing for the country name crosswalk

# Align the CDIAC and World Bank Country Names
country_names = pd.ExcelFile("/Users/nathansuberi/Downloads/country name crosswalk.xlsx").parse("Sheet1")

# Take the two slim sets that match with each other
slim_set = country_names.drop(["Country Names", "Unnamed: 1", "Unnamed: 3",  "Unnamed: 5", "Unnamed: 6"], axis=1)
slim_set.columns = ["cdiac_names", "wb_names"]
slim_set = slim_set.iloc[3:-51]
slim_set.set_index("cdiac_names", inplace=True)

# See Bahamas for an example of a difference
print("Total number of countries:", len(slim_set), "\n")
print("Examples of some country names which differ:\n")
print(slim_set.iloc[38:42])

# Upload CDIAC to World Bank name conversions to RAW_DATA
csv_buffer = io.StringIO()
slim_set.to_csv(csv_buffer)
s3_resource.Object(s3_bucket, CONVERSIONS + \
                   "CDIAC to World Bank name conversion.csv").put(Body=csv_buffer.getvalue())


# Prepare World Bank to ISO3 name conversions

isos = pd.read_csv("/Users/nathansuberi/Desktop/WRI_Programming/world_bank_isos.csv", sep="\n", header=None)
isos = list(isos[0])

pairs = []
for ix, val in enumerate(isos):
    if ix%3==0:
        key = val
    elif ix%3==1:
        value = val
        pairs.append([key,value])

real_isos = pd.DataFrame(pairs, columns=["Country", "ISO"])
real_isos.set_index("Country", inplace=True)

def replace_iso_names(name):
    if name == "Brunei":
        return("Brunei Darussalam")
    elif name == "Cape Verde":
        return("Cabo Verde")
    elif name == "Ethiopia (excludes Eritrea)":
        return("Ethiopia")
    elif name == "Hong Kong, China":
        return("Hong Kong SAR, China")
    elif name == "Macao":
        return("Macao SAR, China")
    elif name == "Venezuela":
        return("Venezuela, RB")
    else:
        return(name)

real_isos.index = map(replace_iso_names, real_isos.index)
real_isos.loc["Montenegro"] = "MNE"
real_isos.loc["Serbia"] = "SRB"
real_isos.loc["Romania"] = "ROM"

# Upload World Bank to ISO3 name conversions to RAW_DATA
csv_buffer = io.StringIO()
real_isos.to_csv(csv_buffer)
s3_resource.Object(s3_bucket, CONVERSIONS + \
                   "World Bank to ISO3 name conversion.csv").put(Body=csv_buffer.getvalue())


Investigate which countries are represented in the data, naming differences

In [46]:
def make_lower(array):
    return([name.lower() for name in array])

subjects1 = make_lower(cdiac_raw_data["Territorial Emissions GCB"].columns.sort_values().values)
subjects2 = make_lower(cdiac_raw_data["Consumption Emissions GCB"].columns.sort_values().values)
subjects3 = make_lower(cdiac_raw_data["Emissions Transfers GCB"].columns.sort_values().values)
subjects4 = make_lower(cdiac_raw_data["Territorial Emissions CDIAC"].columns.sort_values().values)

In [52]:
print(len(subjects1))
print(len(subjects2))
print(len(subjects3))
print(len(subjects4))

# From subjects1
print("\nDifferences between subjects1 and others\n")
print("Not in 2:\n", [name for name in subjects1 if name not in subjects2])
print("Not in 3:\n", [name for name in subjects1 if name not in subjects3])
print("Not in 4:\n", [name for name in subjects1 if name not in subjects4])

# From subjects2
print("\nDifferences between subjects2 and others\n")
print("Not in 1:\n", [name for name in subjects2 if name not in subjects1])
print("Not in 3:\n", [name for name in subjects2 if name not in subjects3])
print("Not in 4:\n", [name for name in subjects2 if name not in subjects4])

# From subjects3
print("\nDifferences between subjects3 and others\n")
print("Not in 1:\n", [name for name in subjects3 if name not in subjects1])
print("Not in 2:\n", [name for name in subjects3 if name not in subjects2])
print("Not in 4:\n", [name for name in subjects3 if name not in subjects4])

# From subjects4
print("\nDifferences between subjects4 and others\n")
print("Not in 1:\n", [name for name in subjects4 if name not in subjects1])
print("Not in 2:\n", [name for name in subjects4 if name not in subjects2])
print("Not in 3:\n", [name for name in subjects4 if name not in subjects3])

235
232
135
235

Differences between subjects1 and others

Not in 2:
 ['bunkers', 'statistical difference', 'world']
Not in 3:
 ['afghanistan', 'algeria', 'andorra', 'angola', 'anguilla', 'antigua and barbuda', 'aruba', 'bahamas', 'barbados', 'belize', 'bermuda', 'bhutan', 'bonaire, saint eustatius and saba', 'bosnia and herzegovina', 'british virgin islands', 'burundi', 'cape verde', 'cayman islands', 'central african republic', 'chad', 'comoros', 'congo', 'cook islands', 'cuba', 'curaçao', 'democratic republic of the congo', 'djibouti', 'dominica', 'equatorial guinea', 'eritrea', 'faeroe islands', 'falkland islands (malvinas)', 'fiji', 'french guiana', 'french polynesia', 'gabon', 'gambia', 'gibraltar', 'greenland', 'grenada', 'guadeloupe', 'guinea-bissau', 'guyana', 'haiti', 'iceland', 'iraq', 'kiribati', 'lebanon', 'lesotho', 'liberia', 'libya', 'liechtenstein', 'macao', 'macedonia (republic of)', 'maldives', 'mali', 'marshall islands', 'martinique', 'mauritania', 'micronesia (fede

Sanity checks

In [152]:
# This shows the total nmber of remaining countries in the data
list1 = consumption_gcb_pct_change.index
list2 = territory_gcb_pct_change.index

print(sum([(True if (item in list2) else False) for item in list1]))
print(sum([(True if (item in list2) else False) for item in list1]))

182
182


In [95]:
### Evaluate whether Null items are the same between Consumption and Territory emissions data
print(sum(pd.isnull(consumption_gcb_pct_change["wb_name"])))
list1 = consumption_gcb_pct_change["wb_name"].loc[pd.isnull(consumption_gcb_pct_change["wb_name"])].index

print(sum(pd.isnull(territory_gcb_pct_change["wb_name"])))
list2 = territory_gcb_pct_change["wb_name"].loc[pd.isnull(territory_gcb_pct_change["wb_name"])].index

# This includes Bunkers and Statistical Difference - reason for extra indices
[(True if (item in list1) else False) for item in list2]

50
52


[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False]

Create ratios of absolute & pct_change territory / consumption emissions

In [123]:
# Convert raw data to percent change by year from 2000 forward

consumption_emissions_abs = consumption_emissions_abs_raw.loc[2000:].transpose()
territorial_emissions_abs = territorial_emissions_abs_raw.loc[2000:].transpose()

terr_over_cons_abs= territorial_emissions_abs.div(consumption_emissions_abs)
# Name for Congo didn't match in the CDIAC data and crosswalk file
terr_over_cons_abs.index = map(replace_congo, terr_over_cons_abs.index)
# Add the wb_name to each dataframe
terr_over_cons_abs["wb_name"] = terr_over_cons_abs.apply(lambda row: fetch_name(row.name), axis=1)
terr_over_cons_abs = terr_over_cons_abs .loc[pd.notnull(terr_over_cons_abs["wb_name"])]
# Only keep the CDIAC data where there is a matching world bank country
terr_over_cons_abs["ISO"] = terr_over_cons_abs.apply(lambda row: add_iso(row.name), axis=1)


# Add in wb_names and ISO codes

In [126]:
terr_over_cons_abs.to_csv(root_folder + "/territorial_emissions_divided_by_consumption_emissions.csv")
terr_over_cons_abs.head(10)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,wb_name,ISO
Albania,0.800594,0.797292,0.754797,0.777339,0.781501,0.722446,0.701079,0.705743,0.720629,0.703902,0.759592,0.835117,0.803945,0.850623,0.852638,NaN,Albania,ALB
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andorra,AND
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Angola,AGO
Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Antigua and Barbuda,ATG
Argentina,0.983844,1.001218,1.343218,1.180393,1.173823,1.135851,1.105193,1.059628,1.052773,1.068888,1.032187,1.015731,0.998175,0.967498,0.975843,NaN,Argentina,ARG
Armenia,0.847929,0.836301,0.819222,0.776985,0.813314,0.785006,0.784651,0.663564,0.800722,0.714944,0.737157,0.823920,0.864960,0.854448,0.891217,NaN,Armenia,ARM
Australia,1.210696,1.209157,1.143223,1.108805,1.096936,1.111065,1.074477,1.051683,1.088941,1.038934,1.040812,1.017913,0.978117,1.012445,1.012342,NaN,Australia,AUS
Austria,0.703480,0.725991,0.739643,0.780930,0.761119,0.755247,0.716800,0.724654,0.734706,0.729986,0.740051,0.697224,0.709868,0.726156,0.711418,NaN,Austria,AUT
Azerbaijan,1.128780,1.088839,0.993185,0.905761,0.885194,0.981340,0.965286,0.951102,0.988968,0.984856,0.986843,0.985668,0.959886,0.925140,0.914798,NaN,Azerbaijan,AZE


In [127]:
territory_gcb = territorial_emissions_abs_raw.loc[1999:2015].transpose().pct_change(axis=1).loc[:,2000:]
consumption_gcb = consumption_emissions_abs_raw.loc[1999:2015].transpose().pct_change(axis=1).loc[:,2000:]

terr_over_cons_per_change= territory_gcb.div(consumption_gcb)
# Name for Congo didn't match in the CDIAC data and crosswalk file
terr_over_cons_per_change.index = map(replace_congo, terr_over_cons_per_change.index)
# Add the wb_name to each dataframe
terr_over_cons_per_change["wb_name"] = terr_over_cons_per_change.apply(lambda row: fetch_name(row.name), axis=1)
terr_over_cons_per_change = terr_over_cons_per_change.loc[pd.notnull(terr_over_cons_per_change["wb_name"])]
# Only keep the CDIAC data where there is a matching world bank country
terr_over_cons_per_change["ISO"] = terr_over_cons_per_change.apply(lambda row: add_iso(row.name), axis=1)


In [128]:
terr_over_cons_per_change.to_csv(root_folder + "/territorial_emissions_divided_by_consumption_emissions_per_change.csv")
terr_over_cons_per_change.head(10)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,wb_name,ISO
Albania,0.299566,0.937917,0.714305,1.297753,0.852608,0.201983,1.501487,8.674855,1.253137,0.034044,-1.878641,3.823670,1.529034,-0.635643,0.613108,NaN,Albania,ALB
Algeria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,DZA
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andorra,AND
Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Angola,AGO
Antigua and Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Antigua and Barbuda,ATG
Argentina,0.581948,0.783871,0.220955,0.357095,0.962319,0.454829,0.732496,-0.035800,0.918788,0.771202,0.543570,0.544728,0.176683,-0.730784,-2.467890,NaN,Argentina,ARG
Armenia,-0.696074,0.609916,1.145786,0.673826,4.041062,0.818478,0.936670,0.424652,-1.082767,1.773853,0.529499,3.836359,1.532735,1.518618,0.127423,NaN,Armenia,ARM
Australia,-0.971258,0.944659,0.167798,0.361899,0.761402,-4.828712,0.294246,0.470558,-0.644656,0.176394,0.754025,-0.378882,0.142905,0.415051,1.010667,NaN,Australia,AUS
Austria,1.078531,2.181483,3.684767,3.327423,0.151761,0.659681,-1.801437,0.768139,0.166104,1.073604,1.255366,-1.042332,0.685523,-0.204181,1.563294,NaN,Austria,AUT
Azerbaijan,-1.009992,-2.315514,0.228219,0.253049,0.664107,-2.013579,0.881145,1.055468,1.373974,1.038411,0.951995,0.985797,0.688276,0.061686,0.798162,NaN,Azerbaijan,AZE


Identifying indicators from Nate's file

In [ ]:
# Grab indicators from Nate's file
indicators = pd.read_csv("/Users/nathansuberi/Desktop/WRI_Programming/compiled independent variable absolute data 1999-2015.csv")

data_names_and_codes = dict(set(zip(indicators["Series Code"], indicators["Series Name"])))
# Remove the nan entry
data_names_and_codes.pop(np.nan)